## Health Care for All Case Study using Pandas

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import statsmodels.api as sm
from scipy.stats import boxcox
pd.options.display.max_rows = 100
## Install xlrd package to load Excel files
#!conda install openpyxl
#!conda install xlrd

### Loading Data

In [2]:
def load_original_data():
    file1 = pd.read_csv('Data/file1.csv')
    file2 = pd.read_csv('Data/file2.txt', sep = '\t')
    file3 = pd.read_excel('Data/file3.xlsx')
    file4 = pd.read_excel('Data/file4.xlsx')
    return pd.concat([file1,file2,file3, file4], axis=0)

In [3]:
hk_df=load_original_data()

In [4]:
hk_df.head()

,CONTROLN,STATE,GENDER,HV1,IC1,IC4,HVP1,IC5,POBC1,POBC2,IC2,IC3,AVGGIFT,TCODE,DOB,DOMAIN,TARGET_D
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.00,365,473.0,0,19387,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.00,301,436.0,3,18837,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.00,252,348.0,0,17991,5,6,280.0,316,6.000000,28,0,C2,5.0


### Make the column names lower case

In [5]:
def lower_case_column_names(hk_df):
    hk_df.columns=[i.lower() for i in hk_df.columns]
    return hk_df

In [6]:
hk_df=lower_case_column_names(hk_df)

In [7]:
hk_df

,controln,state,gender,hv1,ic1,ic4,hvp1,ic5,pobc1,pobc2,ic2,ic3,avggift,tcode,dob,domain,target_d
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.00,365,473.0,0,19387,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.00,301,436.0,3,18837,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.00,252,348.0,0,17991,5,6,280.0,316,6.000000,28,0,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,41537,FL,F,742,396,470.0,0,15356,7,37,424.0,450,14.285714,2,6101,S2,50.0
1002,4881,FL,M,599,100,423.0,4,11670,10,27,406.0,200,9.230769,28,7401,C1,10.0
1003,66358,ND,M,593,258,299.0,0,10186,1,70,298.0,274,5.266667,1,3301,S2,5.0
1004,2231,WI,male,564,217,397.0,0,12315,0,96,386.0,295,11.400000,1,7301,T2,14.0


#### Renaming columns

Columns names can be replaced in several ways. 

One of them is to provide a new **list of column names** and replace the value of `df.columns`

Another is to use the method `.rename()` which can be applied to a dataframe. This method needs as an input a **dictionary** in which the old column names are the keys, while the new column names are the values.



In [8]:
def rename_columns(hk_df):
    hk_df.rename(columns={'controln':'id','hv1':'median_home_val',
                          'ic1':'median_household_income',"ic2":"med_fam_income", 
                          "ic3":"avg_household_income","ic4": "avg_fam_income",
                          "ic5":"per_capita_income"}, inplace=True )
    return hk_df

In [9]:
hk_df=rename_columns(hk_df)

#### Deleting columns

In [10]:
hk_df.dob.value_counts()

0       964
3001     66
2801     61
2401     60
2201     60
       ... 
5005      1
6408      1
7701      1
5311      1
405       1
Name: dob, Length: 626, dtype: int64

In [11]:
hk_df

,id,state,gender,median_home_val,median_household_income,avg_fam_income,hvp1,per_capita_income,pobc1,pobc2,med_fam_income,avg_household_income,avggift,tcode,dob,domain,target_d
0,44060,FL,M,AAA896,392,520.0,7,21975,6,16,430.0,466,28.000000,1,1901,C2,100.0
1,96093,IL,M,537.00,365,473.0,0,19387,1,89,415.0,410,5.666667,0,0,T2,7.0
2,43333,FL,F,725.00,301,436.0,3,18837,11,17,340.0,361,4.111111,0,2501,C2,5.0
3,21885,NC,M,AAA1095,401,413.0,7,14014,1,74,407.0,399,27.277778,0,2208,T2,38.0
4,190108,FL,F,995.00,252,348.0,0,17991,5,6,280.0,316,6.000000,28,0,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,41537,FL,F,742,396,470.0,0,15356,7,37,424.0,450,14.285714,2,6101,S2,50.0
1002,4881,FL,M,599,100,423.0,4,11670,10,27,406.0,200,9.230769,28,7401,C1,10.0
1003,66358,ND,M,593,258,299.0,0,10186,1,70,298.0,274,5.266667,1,3301,S2,5.0
1004,2231,WI,male,564,217,397.0,0,12315,0,96,386.0,295,11.400000,1,7301,T2,14.0


In [12]:
def drop_columns(hk_df) :
    hk_df.drop(columns=["id","tcode",'pobc1',"dob"], inplace=True)
    return hk_df

In [13]:
drop_columns(hk_df)

,state,gender,median_home_val,median_household_income,avg_fam_income,hvp1,per_capita_income,pobc2,med_fam_income,avg_household_income,avggift,domain,target_d
0,FL,M,AAA896,392,520.0,7,21975,16,430.0,466,28.000000,C2,100.0
1,IL,M,537.00,365,473.0,0,19387,89,415.0,410,5.666667,T2,7.0
2,FL,F,725.00,301,436.0,3,18837,17,340.0,361,4.111111,C2,5.0
3,NC,M,AAA1095,401,413.0,7,14014,74,407.0,399,27.277778,T2,38.0
4,FL,F,995.00,252,348.0,0,17991,6,280.0,316,6.000000,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1001,FL,F,742,396,470.0,0,15356,37,424.0,450,14.285714,S2,50.0
1002,FL,M,599,100,423.0,4,11670,27,406.0,200,9.230769,C1,10.0
1003,ND,M,593,258,299.0,0,10186,70,298.0,274,5.266667,S2,5.0
1004,WI,male,564,217,397.0,0,12315,96,386.0,295,11.400000,T2,14.0


#### Rearanging columns

In [14]:
list(hk_df.columns)

['state',
 'gender',
 'median_home_val',
 'median_household_income',
 'avg_fam_income',
 'hvp1',
 'per_capita_income',
 'pobc2',
 'med_fam_income',
 'avg_household_income',
 'avggift',
 'domain',
 'target_d']

Reordering columns is as simple as replacing the dataframe with the new order of the columns.

In [15]:
hk_df = hk_df[[
 'state',
 "gender",
 'median_home_val',
 'avg_fam_income',
 'avg_household_income',
 'med_fam_income',
 'median_household_income',
 'hvp1',
 "per_capita_income",
 'pobc2',
 'avggift',
 'domain',
 'target_d']]

hk_df.head()

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,M,AAA896,520.0,466,430.0,392,7,21975,16,28.000000,C2,100.0
1,IL,M,537.00,473.0,410,415.0,365,0,19387,89,5.666667,T2,7.0
2,FL,F,725.00,436.0,361,340.0,301,3,18837,17,4.111111,C2,5.0
3,NC,M,AAA1095,413.0,399,407.0,401,7,14014,74,27.277778,T2,38.0
4,FL,F,995.00,348.0,316,280.0,252,0,17991,6,6.000000,C2,5.0


Cleaning column gender using map function and another user defined function

In [16]:
hk_df.gender.unique()

array(['M', 'F', 'female', 'Male', nan, 'U', 'J', 'male', 'Female',
       'feamale', 'A'], dtype=object)

In [17]:
def clean_gender(x):
    if x in ['M', 'MALE',"male"]:
        return 'Male'
    elif x in ['F', 'female',"feamale"]:
        return 'Female'
    elif np.nan:  pass
    else:
        return 'U'

In [18]:
hk_df['gender'] = list(map(clean_gender, hk_df['gender'])) 

In [19]:
hk_df.gender.value_counts()

Female    2087
Male      1602
Name: gender, dtype: int64

The method `.describe()` give us a **dataframe** with a statistical summary of every column

In [20]:
hk_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4028 entries, 0 to 1005
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   state                    4028 non-null   object 
 1   gender                   3689 non-null   object 
 2   median_home_val          4026 non-null   object 
 3   avg_fam_income           4027 non-null   float64
 4   avg_household_income     4028 non-null   int64  
 5   med_fam_income           4027 non-null   float64
 6   median_household_income  4028 non-null   int64  
 7   hvp1                     4028 non-null   int64  
 8   per_capita_income        4028 non-null   object 
 9   pobc2                    4028 non-null   int64  
 10  avggift                  4028 non-null   float64
 11  domain                   4028 non-null   object 
 12  target_d                 4028 non-null   float64
dtypes: float64(4), int64(4), object(5)
memory usage: 440.6+ KB


In [22]:
temp = hk_df.describe().transpose()
temp

,count,mean,std,min,25%,50%,75%,max
avg_fam_income,4027.0,444.664515,174.857272,0.00000,325.000000,410.00,528.000000,1331.0
avg_household_income,4028.0,399.788232,164.702061,0.00000,287.000000,365.00,479.250000,1311.0
med_fam_income,4027.0,400.639930,180.377938,0.00000,284.000000,366.00,477.000000,1500.0
median_household_income,4028.0,351.569265,168.316241,0.00000,239.000000,318.00,425.000000,1500.0
hvp1,4028.0,15.961519,28.841221,0.00000,0.000000,1.00,14.000000,99.0
pobc2,4028.0,55.476167,21.510554,0.00000,41.000000,57.00,73.000000,99.0
avggift,4028.0,11.771435,10.136313,2.26087,7.181818,10.00,14.388462,450.0
target_d,4028.0,15.645603,12.672374,1.00000,10.000000,13.46,20.000000,200.0


#### Correcting data types

With `to_numeric` 

In [23]:
hk_df = hk_df.reset_index(drop=True)

In [24]:
hk_df

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,AAA896,520.0,466,430.0,392,7,21975,16,28.000000,C2,100.0
1,IL,Male,537.00,473.0,410,415.0,365,0,19387,89,5.666667,T2,7.0
2,FL,Female,725.00,436.0,361,340.0,301,3,18837,17,4.111111,C2,5.0
3,NC,Male,AAA1095,413.0,399,407.0,401,7,14014,74,27.277778,T2,38.0
4,FL,Female,995.00,348.0,316,280.0,252,0,17991,6,6.000000,C2,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4023,FL,Female,742,470.0,450,424.0,396,0,15356,37,14.285714,S2,50.0
4024,FL,Male,599,423.0,200,406.0,100,4,11670,27,9.230769,C1,10.0
4025,ND,Male,593,299.0,274,298.0,258,0,10186,70,5.266667,S2,5.0
4026,WI,Male,564,397.0,295,386.0,217,0,12315,96,11.400000,T2,14.0


In [25]:
hk_df['median_home_val'] =  pd.to_numeric(hk_df['median_home_val'], errors='coerce')
hk_df['median_household_income'] =  pd.to_numeric(hk_df['median_household_income'], errors='coerce')
hk_df['per_capita_income'] =  pd.to_numeric(hk_df['per_capita_income'], errors='coerce')

In [27]:
corrupted_rows_indices=list(hk_df[hk_df['median_home_val'].isnull()].index)
corrupted_rows_indices
hk_df.iloc[corrupted_rows_indices,:]

,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
0,FL,Male,NaN,520.0,466,430.0,392,7,21975.0,16,28.000000,C2,100.0
3,NC,Male,NaN,413.0,399,407.0,401,7,14014.0,74,27.277778,T2,38.0
8,CA,Female,NaN,250.0,235,206.0,184,0,8708.0,63,8.818182,T2,10.0
9,CA,Female,NaN,617.0,619,NaN,593,61,17838.0,62,6.666667,S1,10.0
35,TX,Male,NaN,531.0,450,457.0,368,5,33797.0,42,12.250000,U1,20.0
3022,CA,Male,NaN,438.0,417,388.0,345,3,15915.0,58,16.142857,C2,20.0
3027,CA,Male,NaN,393.0,376,275.0,257,8,10990.0,51,7.625000,U3,15.0
3036,FL,Female,NaN,464.0,452,415.0,408,11,13050.0,26,20.000000,U2,20.0
3196,CA,Female,NaN,388.0,337,221.0,205,4,22845.0,71,7.454545,R2,12.0
3223,FL,None,NaN,331.0,294,273.0,255,0,13727.0,7,6.600000,C2,7.0


In [28]:
hk_df['avggift'] = hk_df['avggift'].astype('int')

In [29]:
hk_df.select_dtypes('object')

,state,gender,domain
0,FL,Male,C2
1,IL,Male,T2
2,FL,Female,C2
3,NC,Male,T2
4,FL,Female,C2
...,...,...,...
4023,FL,Female,S2
4024,FL,Male,C1
4025,ND,Male,S2
4026,WI,Male,T2


In [30]:
hk_df.select_dtypes('float64')

,median_home_val,avg_fam_income,med_fam_income,per_capita_income,target_d
0,NaN,520.0,430.0,21975.0,100.0
1,537.0,473.0,415.0,19387.0,7.0
2,725.0,436.0,340.0,18837.0,5.0
3,NaN,413.0,407.0,14014.0,38.0
4,995.0,348.0,280.0,17991.0,5.0
...,...,...,...,...,...
4023,742.0,470.0,424.0,15356.0,50.0
4024,599.0,423.0,406.0,11670.0,10.0
4025,593.0,299.0,298.0,10186.0,5.0
4026,564.0,397.0,386.0,12315.0,14.0


In [31]:
hk_df._get_numeric_data()

,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,target_d
0,NaN,520.0,466,430.0,392,7,21975.0,16,28,100.0
1,537.0,473.0,410,415.0,365,0,19387.0,89,5,7.0
2,725.0,436.0,361,340.0,301,3,18837.0,17,4,5.0
3,NaN,413.0,399,407.0,401,7,14014.0,74,27,38.0
4,995.0,348.0,316,280.0,252,0,17991.0,6,6,5.0
...,...,...,...,...,...,...,...,...,...,...
4023,742.0,470.0,450,424.0,396,0,15356.0,37,14,50.0
4024,599.0,423.0,200,406.0,100,4,11670.0,27,9,10.0
4025,593.0,299.0,274,298.0,258,0,10186.0,70,5,5.0
4026,564.0,397.0,295,386.0,217,0,12315.0,96,11,14.0


#### Removing duplicates

We can remove duplicate rows with the method `drop_duplicates()`

In [33]:
hk_df = hk_df.drop_duplicates()

#### Remember to reset the index!!

In [34]:
hk_df.reset_index(inplace=True)

- Dropping columns with more null values
- Replacing / imputing null values
- Removing outliers

<b> dropping columns with more null values -- analyst decides the threshold value

note: that the threshold values are usually higher percentages, for eg if 70% or more values in a column are missing, then its a good idea to drop those columns 


In [35]:
nulls_df = pd.DataFrame(round(hk_df.isna().sum()/len(hk_df),4)*100)
nulls_df = nulls_df.reset_index()
nulls_df.columns = ['header_name', 'percent_nulls']
nulls_df

,header_name,percent_nulls
0,index,0.00
1,state,0.00
2,gender,8.33
3,median_home_val,0.25
4,avg_fam_income,0.03
5,avg_household_income,0.00
6,med_fam_income,0.03
7,median_household_income,0.00
8,hvp1,0.00
9,per_capita_income,0.15


In [36]:
columns_drop = nulls_df[nulls_df['percent_nulls']>1]['header_name']  # dummy case with 3 
print(columns_drop.values)

['gender']


### Replacing / imputing null values

Numerical columns: Some ways to approach the problem

- Ignore these observations
-Replace with general average
-Replace with similar type of averages
-Build model to predict missing values

if you have sufficient hk_df and losing a few rows of hk_df is not too bad, you can filter those
rows out as well

In [37]:
hk_df[hk_df['gender'].isna()==True] # checking rows that are null based on a specific column 

,index,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d
13,13,AP,None,0.0,0.0,0,0.0,0,0,0.0,0,5,0,6.0
15,15,MI,None,890.0,523.0,512,474.0,463,1,9493.0,53,11,S1,15.0
19,19,CA,None,1951.0,625.0,619,578.0,556,44,17208.0,66,9,S1,12.0
21,21,OK,None,1040.0,656.0,579,609.0,472,0,26962.0,56,11,C2,15.0
22,22,AR,None,603.0,341.0,288,264.0,208,2,12038.0,70,5,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3944,3968,MI,None,928.0,527.0,403,463.0,334,1,23732.0,57,15,S3,25.0
3958,3982,AL,None,367.0,276.0,236,220.0,179,0,9389.0,79,11,R3,20.0
3964,3988,FL,None,599.0,288.0,253,236.0,200,2,11547.0,34,7,T2,5.0
3969,3993,TX,None,1101.0,640.0,609,600.0,538,0,19641.0,48,16,S1,25.0


In [39]:
hk_df = hk_df[hk_df['med_fam_income'].isna()==False] # Since we don't have many NaN's, we can filter them 
hk_df = hk_df[hk_df['avg_fam_income'].isna()==False]
hk_df = hk_df[hk_df['per_capita_income'].isna()==False]

In [40]:
mean_median_home_value = np.mean(hk_df['median_home_val'])
hk_df['median_home_val'] = hk_df['median_home_val'].fillna(mean_median_home_value)

Replacing null values for categorical variables 

General Approaches:
    
- Ignore observation
- Replace by most frequent value
- Replace using an algorithm like KNN using the neighbours.
- Predict the observation using a multiclass predictor
- Treat missing data as just another category



In [41]:
hk_df['gender'].value_counts()

Female    2066
Male      1592
Name: gender, dtype: int64

In [42]:
len(hk_df[hk_df['gender'].isna()==True])  # number of missing values

333

In [43]:
hk_df['gender'] = hk_df['gender'].fillna('F')

In [44]:
hk_df['gender'].unique() # check the unique values in the column

array(['Male', 'Female', 'F'], dtype=object)

### Removing outliers 
This is done using a box plot. After identifying the upper limit and the lower limit values for a numerical column, we can use filters to remove those rows from the dataframe

### Creating buckets / groups of data  ( Binning)

In [45]:
pd.cut(hk_df['med_fam_income'],4) # to check the bins

0       (375.0, 750.0]
1       (375.0, 750.0]
2        (-1.5, 375.0]
3       (375.0, 750.0]
4        (-1.5, 375.0]
             ...      
3994     (-1.5, 375.0]
3995     (-1.5, 375.0]
3996    (375.0, 750.0]
3997    (375.0, 750.0]
3998     (-1.5, 375.0]
Name: med_fam_income, Length: 3991, dtype: category
Categories (4, interval[float64, right]): [(-1.5, 375.0] < (375.0, 750.0] < (750.0, 1125.0] < (1125.0, 1500.0]]

In [46]:
med_fam_income_labels = ['Low', 'Moderate', 'High', 'Very High']
hk_df['med_fam_income_binned'] = pd.cut(hk_df['med_fam_income'],4, labels=med_fam_income_labels)

In [47]:
hk_df

,index,state,gender,median_home_val,avg_fam_income,avg_household_income,med_fam_income,median_household_income,hvp1,per_capita_income,pobc2,avggift,domain,target_d,med_fam_income_binned
0,0,FL,Male,1158.598192,520.0,466,430.0,392,7,21975.0,16,28,C2,100.0,Moderate
1,1,IL,Male,537.000000,473.0,410,415.0,365,0,19387.0,89,5,T2,7.0,Moderate
2,2,FL,Female,725.000000,436.0,361,340.0,301,3,18837.0,17,4,C2,5.0,Low
3,3,NC,Male,1158.598192,413.0,399,407.0,401,7,14014.0,74,27,T2,38.0,Moderate
4,4,FL,Female,995.000000,348.0,316,280.0,252,0,17991.0,6,6,C2,5.0,Low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3994,4018,MI,Female,632.000000,388.0,339,336.0,279,2,12653.0,71,8,0,5.0,Low
3995,4019,FL,Male,595.000000,274.0,262,263.0,252,0,11132.0,11,14,T2,20.0,Low
3996,4020,CA,Female,2707.000000,537.0,538,504.0,507,80,16165.0,54,12,U1,22.0,Moderate
3997,4021,CA,Male,2666.000000,653.0,612,609.0,535,63,24745.0,45,12,S1,21.0,Moderate


Exercise:<br>
<b>For cities create a user defined function and use it with map() to divide the cities into 5 zones - East, west, north, south, and central

#### Summary functions

sum(), mean(), max()...

In [48]:
hk_df.median_household_income.min()

0

In [49]:
hk_df["median_home_val"].sum()

4623965.38372677

In [50]:
np.sum(hk_df["median_home_val"])

4623965.38372677

#### Saving the df to a csv

In [51]:
hk_df.to_csv("hk_df_cleaned.csv",index=False)